Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

In [1]:
from random import choices
from dataclasses import dataclass
from typing import List
from tqdm import tqdm
from random import random
import numpy as np

import lab9_lib

In [2]:
POPULATION_SIZE = 50
OFFSPRING_SIZE = 25
EXTINCTION_SIZE = 20
GENERATIONS = 10000
CROSSOVER_PROBABILITY = 0.8
MUTATION_PROBABILITY = 0.5
CUTS = 5
TOURNAMENT_SIZE = 3


In [3]:
@dataclass
class Individual:
  genotype: List[int]
  fitness: float = 0.0

  def crossover(self, ff, other: "Individual", cuts: int = CUTS) -> "Individual":
    """Returns two new individuals from crossing self and other and picks the best one."""
    assert len(self.genotype) == len(other.genotype)
    resulting_genotype_1 = []
    resulting_genotype_2 = []
    cuttings = choices(range(len(self.genotype)), k=cuts)
    cuttings.sort()
    resulting_genotype_1 += self.genotype[:cuttings[0]]
    resulting_genotype_2 += other.genotype[:cuttings[0]]
    if cuts == 1:
      resulting_genotype_1 += other.genotype[cuttings[0]:]
      resulting_genotype_2 += self.genotype[cuttings[0]:]
      individual_1 = Individual(fitness=ff(resulting_genotype_1), genotype=resulting_genotype_1)
      individual_2 = Individual(fitness=ff(resulting_genotype_2), genotype=resulting_genotype_2)
      return individual_1 if individual_1.fitness > individual_2.fitness else individual_2
    else:
      for i in range(1, len(cuttings)):
        if i % 2 == 0:
          resulting_genotype_1 += self.genotype[cuttings[i-1]:cuttings[i]]
          resulting_genotype_2 += other.genotype[cuttings[i-1]:cuttings[i]]
        else:
          resulting_genotype_1 += other.genotype[cuttings[i-1]:cuttings[i]]
          resulting_genotype_2 += self.genotype[cuttings[i-1]:cuttings[i]]
      resulting_genotype_1 += self.genotype[cuttings[-1]:]
      resulting_genotype_2 += other.genotype[cuttings[-1]:]
      individual_1 = Individual(fitness=ff(resulting_genotype_1), genotype=resulting_genotype_1)
      individual_2 = Individual(fitness=ff(resulting_genotype_2), genotype=resulting_genotype_2)

      return individual_1 if individual_1.fitness > individual_2.fitness else individual_2

  def mutate(self, ff) -> "Individual":
    """Returns a new individual from mutating self."""
    resulting_genotype = []
    changes = choices(range(len(self.genotype)), k=50)
    for i in range(len(self.genotype)):
      if i in changes:
        resulting_genotype.append(1 - self.genotype[i])
      else:
        resulting_genotype.append(self.genotype[i])
    return Individual(fitness=ff(resulting_genotype), genotype=resulting_genotype)
    

  def __repr__(self):
    return f"Individual(fitness={self.fitness:.2%}, genotype={self.genotype})"



  
    

In [4]:

def begin_problem(problem_size: int = 1) -> List[Individual]:
  fitness = lab9_lib.make_problem(problem_size)
  population = [Individual(genotype = choices([0, 1], k=1000)) for _ in range(POPULATION_SIZE)]
  for i in range(POPULATION_SIZE):
   population[i].fitness = fitness(population[i].genotype)
  return population, fitness


def parent_selection(population: List[Individual], type: str = "Random") -> List[Individual]:
  """Returns a list of individuals from the population that will be used for crossover."""
  if type == "Random":
    return choices(population, k=2)
  elif type == "Tournament":
    contestants = choices(population, k=TOURNAMENT_SIZE)
    contestants.sort(key=lambda x: x.fitness, reverse=True)

    contestants2 = choices(population, k=TOURNAMENT_SIZE)
    contestants2.sort(key=lambda x: x.fitness, reverse=True)
    return contestants[0], contestants2[0]
  elif type == "Roulette":
    return choices(population, weights=[x.fitness for x in population], k=3)
  else:
    raise ValueError("Invalid parent selection type")

  
    


In [5]:
epochs_bar = tqdm(range(GENERATIONS), unit="epoch", desc="Epochs")
population, fitness = begin_problem(2)
epoch_stddev = []
for epoch in epochs_bar:
  if epoch % 10 == 0:
    epoch_stddev.append(np.std([x.fitness for x in population]))
  if epoch % 100 == 0:
    epoch_stddev.sort()
    if epoch_stddev[-1] - epoch_stddev[0] < 0.01:
      population = population[EXTINCTION_SIZE:]
      population.extend([Individual(genotype = choices([0, 1], k=1000)) for _ in range(EXTINCTION_SIZE)])
      for i in range(EXTINCTION_SIZE):
        population[i].fitness = fitness(population[i].genotype)
      epoch_stddev = []
  new_population = []
  for _ in range(OFFSPRING_SIZE):
    parent1, parent2 = parent_selection(population, "Tournament")
    if random() < MUTATION_PROBABILITY:
      child = parent1.mutate(fitness) if parent1.fitness > parent2.fitness else parent2.mutate(fitness)
    if random() < CROSSOVER_PROBABILITY:
      child = parent1.crossover(fitness, parent2)
    new_population.append(child)
  population.extend(new_population)
  population.sort(key=lambda x: x.fitness, reverse=True)
  population = population[:POPULATION_SIZE]
  epochs_bar.set_postfix({"fitness": population[0].fitness})
  

Epochs:   0%|          | 0/10000 [00:00<?, ?epoch/s, fitness=0.54] 

Epochs: 100%|██████████| 10000/10000 [01:16<00:00, 131.02epoch/s, fitness=0.7] 


In [6]:
fitness.calls

525073